This notebook follows `munic_model_prod.ipynb`, where we ran the model. Here, we'll extract and format the out-of-sample data, to then push them through the model and get predictions for Paris 2020 city-council elections. Then, we'll analyze and plot the results in the notebook `munic_model_analysis.ipynb`. You don't have to understand the model to read this notebook, but if you're curious about it, please go ahead and read the other notebook!

So, as usual, let's start by importing the necessary packages and defining handy helper functions:

In [1]:
import arviz as az
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pymc3 as pm
import xarray as xr

from bokeh.palettes import brewer
from scipy.special import softmax
from typing import List

In [2]:
az.style.use("arviz-darkgrid")
SPAN_POLLS = 5
ALPHA_POLLS = 2 / (SPAN_POLLS + 1)
CANDIDATES = {
    "Simonnet": "farleft",
    "Hidalgo": "left",
    "Belliard": "green",
    "Buzyn": "center",
    "Griveaux": "center",
    "Dati": "right",
    "Federbusch": "farright",
}
MONTHS = {"janvier": 1, "février": 2, "mars": 3}
PARTIES = ["farleft", "left", "green", "center", "right", "farright", "other"]
Nparties = len(PARTIES) - 1
PARTIES_AGG = [
    "farleft_agg",
    "left_agg",
    "green_agg",
    "center_agg",
    "right_agg",
    "farright_agg",
]
RIGHT_POLLSTER = {
    "Harris Interactive": "Harris",
    "Ifop-Fiducial": "Ifop",
    "Ipsos-Sopra Steria": "Ipsos",
    "Ipsos-Sopra Steria[87]": "Ipsos",
    "Ipsos- Sopra [90]Steria": "Ipsos"
}
BINS = np.array([15.0, 25.0, 35.0, 45.0, 55.0, 65.0, 75.0])
COLORS = {
    "farleft": np.array(brewer["Reds"][7][::-1]),
    "left": np.array(brewer["PuRd"][7][::-1]),
    "green": np.array(brewer["Greens"][7][::-1]),
    "center": np.array(brewer["Oranges"][7][::-1]),
    "right": np.array(brewer["Blues"][7][::-1]),
    "farright": np.array(brewer["Purples"][7][::-1]),
    "other": np.array(brewer["Greys"][7][::-1]),
}

In [3]:
def compute_analyt_weights(df: pd.DataFrame) -> pd.DataFrame:

    pollster_ratings = pd.read_csv("../data/polls_1st_round/pollsters_weights.csv")
    df = pd.merge(
        df, pollster_ratings, how="left", left_on="pollster", right_on="sondage"
    )

    for p in PARTIES[:-1]:
        df[f"weightsondeur_{p}"].fillna(
            pollster_ratings[f"weightsondeur_{p}"].median(), inplace=True
        )
        df[f"analyt_weights_{p}"] = np.log(df.samplesize) * df[f"weightsondeur_{p}"]

    return df.set_index("date").sort_index()


def agg_polls(df: pd.DataFrame) -> pd.DataFrame:

    unique_dates = sorted(set(df.index))

    for p in PARTIES[:-1]:
        for i, d_outer in enumerate(unique_dates):
            for j, d_inner in enumerate(unique_dates[: i + 1]):
                df.loc[d_inner, f"expon_weights_{p}"] = (1 - ALPHA_POLLS) ** (i - j)

            df[f"final_weights_{p}"] = (
                df[f"analyt_weights_{p}"] * df[f"expon_weights_{p}"]
            )
            final_weights = df.loc[:d_outer, f"final_weights_{p}"]
            vote_share = df.loc[:d_outer, f"{p}"]

            df.loc[d_outer, f"{p}_agg"] = np.average(vote_share, weights=final_weights)

            # compute aggregate sample size only once:
            if p == "right":
                # same weights, whatever the party:
                expon_weights = df.loc[:d_outer, "expon_weights_right"]
                sample_size = df.loc[:d_outer, "samplesize"]
                df.loc[d_outer, "samplesize_agg"] = round(
                    np.average(sample_size, weights=expon_weights)
                )

    return df.reset_index()[
        ["date", "samplesize_agg"] + [f"{p}_agg" for p in PARTIES[:-1]]
    ]

Now, let's load our MCMC samples and data -- nothing really thrilling here:

In [4]:
# posterior samples
trace_prod = az.from_netcdf("trace_prod.nc")
post = trace_prod.posterior
trace_prod

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data
	> constant_data

In [5]:
# test data
unemp = pd.read_excel(
    "../data/predictors/chomage-zone-demploi-2003-2019.xls",
    header=5,
    sheet_name="txcho_ze",
)
unemp = unemp[unemp["LIBZE2010"] == "Paris"].iloc[:, 4:].T
unemp.columns = ["unemployment"]
unemp.index = pd.period_range(start=unemp.index[0], periods=len(unemp), freq="Q")
unemp

,unemployment
2003Q1,8.4
2003Q2,8.7
2003Q3,8.6
2003Q4,9.0
2004Q1,9.2
...,...
2018Q3,7.7
2018Q4,7.4
2019Q1,7.4
2019Q2,7.2


In [6]:
# training data
d = pd.read_csv("../data/whole_formatted_data.csv", index_col=0)
district_id, districts = d.arrondissement.factorize(sort=True)
Ndistricts = len(districts)
type_id, types = d.type.factorize(sort=True)

Here is something more fun: we're going to scrape the most recent polls from the appropriate [Wikipedia page](https://fr.wikipedia.org/wiki/%C3%89lections_municipales_de_2020_%C3%A0_Paris). Indeed, our model is trained on polls and unemployment data from previous elections. To get predictions for the coming city-council elections (March 15th, 2020), we need the last [unemployment figures in Paris](https://www.insee.fr/fr/statistiques/1893230) (most recent are for Q3 2019) and the last polls. Here is how the scraping of polls goes:

In [7]:
raw_polls = pd.read_html(
    "https://fr.wikipedia.org/wiki/%C3%89lections_municipales_de_2020_%C3%A0_Paris",
    attrs={"class": "wikitable centre"},
    match="Date de réalisation",
    decimal=",",
    thousands=" ",
    na_values=["—", "?"],
)[0]
raw_polls.columns = raw_polls.columns.droplevel([0, 2])
raw_polls = raw_polls[
    ~raw_polls.Source.str.contains("candidature|annonce|renonce|retire|command", regex=True)
].drop(["Gantzer puis aucun chef de file", "Villani", "Bournazel", "Campion", "Berkani", "Autres"], axis=1)

# clean polls' characteristics:
raw_polls = raw_polls.rename(
    columns={
        "Source": "pollster",
        "Date de réalisation": "date",
        "Échantillon": "samplesize",
    }
)
raw_polls["pollster"] = raw_polls.pollster.replace(RIGHT_POLLSTER)

# drop any poll whose sample size is nan, then cast as int
raw_polls = raw_polls.dropna(subset=["samplesize"])
raw_polls["samplesize"] = raw_polls["samplesize"].str.split().str.join("").astype(int)

# take last field date:
field = raw_polls["date"].str.split(" au ", expand=True)[1].str.split(expand=True)
field.columns = ["day", "month"]
field["month"] = field["month"].replace(MONTHS)
field["year"] = 2020
raw_polls["date"] = pd.to_datetime(field[["day", "month", "year"]])

# clean candidates' values:
raw_polls[list(CANDIDATES.keys())] = raw_polls[CANDIDATES.keys()].astype(float)
raw_polls["Buzyn"] = raw_polls[["Buzyn", "Griveaux"]].fillna(0).sum(axis=1)
raw_polls = (
    raw_polls.drop("Griveaux", axis=1)
    .rename(columns=CANDIDATES)
    .sort_values("date")
    .dropna()
    .reset_index(drop=True)
)
raw_polls.to_csv("oos_data/raw_polls_2020.csv")
raw_polls

,pollster,date,samplesize,farleft,left,green,center,right,farright
0,Ifop,2020-01-17,955,5.0,25.0,14.0,15.0,19.0,5.0
1,Ifop,2020-01-17,955,5.0,25.0,14.0,16.0,17.0,5.0
2,Odoxa,2020-01-20,879,8.0,24.0,13.0,16.0,18.0,5.0
3,Odoxa,2020-01-23,916,4.0,23.0,14.5,16.0,20.0,6.0
4,Ipsos,2020-02-19,1000,5.0,24.0,13.0,19.0,20.0,4.0
5,Harris,2020-02-19,1092,6.0,23.0,13.0,17.0,23.0,5.0
6,Odoxa,2020-02-19,809,7.0,23.0,14.0,17.0,25.0,4.0
7,Ifop,2020-02-21,976,6.0,24.0,12.0,19.0,22.0,3.5
8,Ifop,2020-02-28,946,5.0,24.0,11.0,20.0,25.0,3.5
9,Elabe,2020-02-28,1001,5.0,24.0,9.5,18.5,25.0,4.0


Pretty nice, uh? Yeah, pandas is awesome! Now, we're going to aggregate those polls by recency, sample size and historical performance of the pollster. This last weight is based on [a pollster ratings I computed](https://www.pollsposition.com/indicateurs/pollster_ratings) and updated with last year's polls (2019 European elections) -- I didn't have time to open-source this analysis yet, but hopefully one day I will! I'm not going to detail everything by writing, but you can see how it's done in the code of the two functions we defined at the beginning -- `compute_analyt_weights` and `agg_polls`. The code below also transforms the polls from their natural habitat ($[0, 1]$) to the real line ($[-\infty, +\infty]$. This is for technical reasons that I'm not going to detail here -- if you're curious, I explain everything in the notebook of the model.

In [9]:
raw_polls = pd.read_csv("oos_data/raw_polls_2020.csv", index_col=0)
oos_polls = compute_analyt_weights(raw_polls)
oos_polls = agg_polls(oos_polls).drop_duplicates()
# revert the softmax:
oos_polls[PARTIES_AGG] = oos_polls[PARTIES_AGG].div(100).apply(np.log) + 1
oos_polls.round(2)

,date,samplesize_agg,farleft_agg,left_agg,green_agg,center_agg,right_agg,farright_agg
0,2020-01-17,955.0,-2.00,-0.39,-0.97,-0.86,-0.71,-2.00
2,2020-01-20,922.0,-1.83,-0.40,-0.99,-0.85,-0.71,-2.00
3,2020-01-23,920.0,-1.94,-0.43,-0.97,-0.84,-0.67,-1.92
4,2020-02-19,950.0,-1.87,-0.45,-1.01,-0.75,-0.55,-2.07
7,2020-02-21,956.0,-1.85,-0.44,-1.03,-0.73,-0.54,-2.12
8,2020-02-28,963.0,-1.91,-0.44,-1.14,-0.70,-0.47,-2.18
10,2020-03-02,1001.0,-1.94,-0.43,-1.16,-0.71,-0.45,-2.19
11,2020-03-06,1043.0,-2.03,-0.42,-1.14,-0.69,-0.43,-2.20
13,2020-03-09,1021.0,-2.05,-0.41,-1.16,-0.68,-0.44,-2.21
14,2020-03-10,1042.0,-2.03,-0.39,-1.17,-0.68,-0.44,-2.23


Ok, we're ready to make out-of-sample predictions! This is going to look weird and esoteric, but I'm actually just taking each parameters for the couples (district, party) and pushing them through the model to get the posterior probabilities of each party, in each district -- this will make more sense when we get to the vizualization part.

The interesting part though is that we're gonna use ArviZ's `InferenceData` capabilities instead of raw numpy arrays. That way, we won't have to take care shape broadcasting, adding new axes, removing axes, and similar issues that usually arise. And when I say "we", it's actually Oriol Abril-Pla who helped me tremedously to code this part. So, let's sincerely thank Oriol and [check out his great blog](https://oriolabril.github.io/oriol_unraveled/) ;)

Let's begin by defining some variables and helper functions, to make our code more readable:

In [10]:
# standardize latest polls:
last_polls = (oos_polls[PARTIES_AGG].iloc[-1] - d[PARTIES_AGG].stack().mean()) / d[PARTIES_AGG].stack().std()
last_polls.values

array([-0.02300336,  0.37182675,  0.18305373,  0.30088352,  0.35941297,
       -0.08042145])

In [11]:
# standardize latest unemployment:
last_unemployment = (
    (np.log(unemp.iloc[-1]) - np.log(d["unemployment"]).mean())
    / np.log(d["unemployment"]).std()
).iloc[0]
last_unemployment

-1.0281829564033451

In [12]:
trace_prod.add_groups(predictions_constant_data=
    xr.Dataset(
        {
            "incumbent_indicator": (["district", "incumbents"], np.random.multinomial(1, [0.35, 0.30, 0.35], size=Ndistricts)),
            "stdz_polls": (["party"], last_polls.values),
            "log_unemployment": ([], last_unemployment),
        },
        coords={
            "district": post.district.values,
            "incumbents": trace_prod.constant_data.incumbents.values,
            "party": trace_prod.constant_data.party.values,
        },
    )
)
const = trace_prod.predictions_constant_data
trace_prod

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data
	> constant_data
	> predictions_constant_data

In [19]:
def stdz(series: pd.Series) -> pd.Series:
    """Standardize the given pandas Series"""
    return (series - series.mean()) / series.std()

In [22]:
# CHANGE THE STANDARDIZATION OF POLLS IN MODEL NOTEBOOK
stdz(d[PARTIES_AGG]).describe().round(2)

,farleft_agg,left_agg,green_agg,center_agg,right_agg,farright_agg
count,221.00,221.00,221.00,221.00,221.00,221.00
mean,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00
std,1.00,1.00,1.00,1.00,1.00,1.00
min,-2.05,-2.11,-3.42,-2.33,-2.25,-2.43
25%,-0.63,-0.26,0.19,0.31,-0.77,-0.55
50%,0.04,0.30,0.31,0.41,0.10,0.32
75%,0.77,0.60,0.37,0.49,0.75,0.86
max,1.67,1.27,0.47,0.55,1.59,1.22


In [23]:
ps = {}
for p in PARTIES_AGG:
    ps[p] = stdz(d[p])

In [29]:
pd.concat(ps, axis=1).describe().round(2)

,farleft_agg,left_agg,green_agg,center_agg,right_agg,farright_agg
count,221.00,221.00,221.00,221.00,221.00,221.00
mean,-0.00,0.00,-0.00,0.00,0.00,-0.00
std,1.00,1.00,1.00,1.00,1.00,1.00
min,-2.05,-2.11,-3.42,-2.33,-2.25,-2.43
25%,-0.63,-0.26,0.19,0.31,-0.77,-0.55
50%,0.04,0.30,0.31,0.41,0.10,0.32
75%,0.77,0.60,0.37,0.49,0.75,0.86
max,1.67,1.27,0.47,0.55,1.59,1.22


In [13]:
def prepend_strings(base: str, strings: List[str]) -> List[str]:
    return [f"{base}_{item}" for item in strings]

def ds_to_da_list(ds: xr.Dataset , var_names: List[str]) -> List[xr.DataArray]:
    """Turn part of an xarray data set into a list of xarray data arrays"""
    return [ds[name] for name in var_names]

In [14]:
# step 1: reorganize variables to avoid loops
# handle the party loops
post["poll_diff"] = xr.concat(ds_to_da_list(post, prepend_strings("poll_diff", post.party.values)), "party")
post["type_effect"] = xr.concat(ds_to_da_list(post, prepend_strings("type_effect", post.party.values)), "party")

In [15]:
# align party and params dims
post["β_district"] = xr.concat(
    [
        da.rename({"params_short" if "params_short" in da.dims else "params_extend": "params"})
        for da in ds_to_da_list(post, prepend_strings("β_district", post.party.values))
    ], 
    "party",
    fill_value=0
)

# align incumbent with party dimension
_, incumbent_indicator = xr.align(
    post["party"],
    const["incumbent_indicator"].rename(incumbents="party"),
    join="left",
    fill_value=0
)

In [16]:
# broadcasting city-level polls to each district
_, poll_broadcast = xr.broadcast(
    post["district"],
    const["stdz_polls"],
)
# add noise due to individual polling errors
poll_uncert = xr.apply_ufunc(lambda poll, poll_std: np.random.normal(poll, poll_std), poll_broadcast, post["poll_diff"].std(("chain", "draw", "obs_id")))
poll_uncert

<xarray.DataArray (district: 17, party: 6)>
array([[ 0.10080379,  0.39063505,  0.11115024,  0.44423038,  0.37854704,
         0.19608492],
       [-0.05548205,  0.42588202,  0.23671021,  0.2025311 ,  0.34149934,
        -0.10224672],
       [-0.20760891,  0.29754011,  0.21078113,  0.32429955,  0.42147716,
        -0.09093633],
       [ 0.06859433,  0.3177638 ,  0.05045902,  0.41661339,  0.28742234,
        -0.17562518],
       [ 0.0405347 ,  0.37491591,  0.10472219,  0.17017109,  0.35167975,
         0.02033863],
       [-0.03551606,  0.35555893,  0.18618321,  0.43286408,  0.36883998,
        -0.18175358],
       [-0.02760177,  0.4363333 ,  0.06458947,  0.20558306,  0.39151049,
        -0.1502354 ],
       [-0.16942889,  0.36119088,  0.14028843,  0.35728063,  0.44312513,
        -0.12075068],
       [-0.01109057,  0.23193118,  0.24274954,  0.28629565,  0.44819298,
        -0.0088929 ],
       [ 0.1013759 ,  0.27204594,  0.26226805,  0.22371079,  0.31661898,
         0.04650991],
       [-0.06558124,  0.43976577,  0.27955568,  0.34295949,  0.40356781,
        -0.08462329],
       [ 0.07211151,  0.41440674,  0.16820309,  0.4299922 ,  0.26972006,
        -0.0683557 ],
       [-0.01253032,  0.34342642,  0.36239085,  0.15092066,  0.32936855,
        -0.11841531],
       [ 0.08576417,  0.35690443,  0.15231311,  0.33590652,  0.24947028,
        -0.03498882],
       [-0.09420313,  0.29602874,  0.17858006,  0.29042424,  0.3382158 ,
         0.0163706 ],
       [ 0.17279997,  0.3785013 ,  0.20453945,  0.38544113,  0.27603423,
        -0.18448132],
       [-0.04622335,  0.40888736, -0.03356549,  0.26871803,  0.38298809,
         0.06016312]])
Coordinates:
  * party     (party) object 'farleft' 'left' 'green' ... 'right' 'farright'
  * district  (district) int64 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20

In [17]:
# step 2: remove unnecessary vars from memory, to relieve RAM
# use dask?
#post.drop([
   # *prepend_strings("poll_diff", post.party.values),
  #  *prepend_strings("type_effect", post.party.values),
 #   *prepend_strings("β_district", post.party.values)
#]) 

# step 3: write the calculation mus_parties
mus_parties = (
    poll_uncert
    + post["β_district"].sel(params="local_standing")
    + post["type_effect"].sel(election_type="municipale")
    + post["β_district"].sel(params="log_unemployment") * const["log_unemployment"]
    + post["β_district"].sel(params="incumbency") * incumbent_indicator
).drop("params")
mus_parties

<xarray.DataArray (district: 17, party: 6, chain: 8, draw: 2000)>
array([[[[-2.09744661, -1.87267289, -2.09811933, ..., -1.8459369 ,
          -2.00552701, -1.87993582],
         [-2.00977027, -2.06330951, -1.96834108, ..., -1.87937288,
          -1.93099396, -1.8468866 ],
         [-1.94187006, -1.96261211, -1.91095318, ..., -1.92790674,
          -1.92951775, -1.79450694],
         ...,
         [-1.87381335, -2.03782198, -1.96980891, ..., -1.72590327,
          -1.85453858, -1.8519313 ],
         [-1.88557935, -1.94692005, -1.91353848, ..., -1.9502508 ,
          -1.95523699, -1.92534572],
         [-1.87212021, -1.89143919, -1.86103384, ..., -1.90908279,
          -1.92328458, -2.00305164]],

        [[-1.02501076, -1.06022062, -1.0021765 , ..., -1.01289972,
          -1.0383512 , -1.03443618],
         [-1.04749781, -1.13256449, -0.9836279 , ..., -1.04964275,
          -0.87589618, -0.95057945],
         [-0.94757775, -1.03564332, -0.98822152, ..., -0.96241699,
          -1.01286639, -1.16175132],
...
         [-3.28253147, -3.28104656, -3.13929435, ..., -3.21296215,
          -3.27883907, -3.32878599],
         [-3.22843474, -3.30004883, -3.30508382, ..., -3.32563643,
          -3.27436271, -3.35255862],
         [-3.29436007, -3.28023246, -3.27575621, ..., -3.38028264,
          -3.38092222, -3.27295625]],

        [[-2.20741874, -2.16901748, -2.22388149, ..., -2.22070311,
          -2.27973521, -2.34484759],
         [-2.20115636, -2.1593838 , -2.26188994, ..., -2.32085855,
          -2.25973604, -2.31159696],
         [-2.20308444, -2.28298026, -2.22000908, ..., -2.31412332,
          -2.17809338, -2.4077974 ],
         ...,
         [-2.23983303, -2.24132992, -2.20462525, ..., -2.40726906,
          -2.25494067, -2.29619079],
         [-2.17527135, -2.33537899, -2.324104  , ..., -2.23415153,
          -2.26836304, -2.24660259],
         [-2.28936496, -2.26036945, -2.36064042, ..., -2.38996012,
          -2.2867844 , -2.24217583]]]])
Coordinates:
  * party          (party) object 'farleft' 'left' ... 'right' 'farright'
  * district       (district) int64 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
  * chain          (chain) int64 0 1 2 3 4 5 6 7
  * draw           (draw) int64 0 1 2 3 4 5 6 ... 1994 1995 1996 1997 1998 1999
    election_type  <U10 'municipale'

In [18]:
def add_noise_and_other_party(mu_parties, noise, vary_piv_mu, vary_piv_std, rng=None):
    """
    Implement last steps of the model: 
        - Add systematic noise around the party's estimates
        - Sample the last category from the posterior and concatenate it to the other parties
        - Softmax everything to get vote shares
    Parameters
    ----------
    mu_parties: posterior shares for the six parties
    noise: posterior noise of systematic errors for each party
    vary_piv_mu: posterior mean of last multinomial category
    vary_piv_std: posterior std of last multinomial category
    """
    # account for systematic polling errors, media narratives, “October surprises":
    noisy_mus = mu_parties + rng.normal(scale=noise)
    
    # append last category:
    vary_pivot = rng.normal(loc=vary_piv_mu, scale=vary_piv_std, size=(*noisy_mus.shape[:-1], 1))
    noisy_post_preds = np.concatenate((noisy_mus, vary_pivot), axis=-1)
    
    # preferences of each district
    share_est = softmax(noisy_post_preds, axis=-1) * 100
    
    return share_est

In [19]:
# step 4-6: 
#     add noise (noisy mus)
#     add vary_pivot and concat
#     apply softmax along party dimension

share_est = xr.apply_ufunc(
    add_noise_and_other_party,
    mus_parties,
    post["noisy_mus"].std(("chain", "draw", "obs_id")),
    post["vary_pivot"].mean(), 
    post["vary_pivot"].std(),
    kwargs={"rng": np.random.default_rng()},
    input_core_dims=[["party"], ["party"], [], []],
    output_core_dims=[["party_complete"]],
).assign_coords(party_complete=post.party_complete)

In [20]:
share_est.to_dataset(name="share_est") # dataset views are more compact and informative than dataarray ones

<xarray.Dataset>
Dimensions:         (chain: 8, district: 17, draw: 2000, party_complete: 7)
Coordinates:
  * district        (district) int64 4 5 6 7 8 9 10 11 ... 14 15 16 17 18 19 20
  * chain           (chain) int64 0 1 2 3 4 5 6 7
  * draw            (draw) int64 0 1 2 3 4 5 6 ... 1994 1995 1996 1997 1998 1999
    election_type   <U10 'municipale'
  * party_complete  (party_complete) object 'farleft' 'left' ... 'other'
Data variables:
    share_est       (district, chain, draw, party_complete) float64 2.151 ......

In [21]:
share_est.sel(district=20, draw=slice(5)) #.idxmax(dim="party_complete")#.groupby(...).count()

<xarray.DataArray (chain: 8, draw: 6, party_complete: 7)>
array([[[ 5.43029776, 35.49342177,  8.66314379,  0.92481664,
          4.29119483, 28.8632041 , 16.3339211 ],
        [ 6.02941667, 39.94520787,  9.94246229,  1.02751438,
          3.84398698, 31.05290186,  8.15850996],
        [ 5.77386574, 38.88954133,  9.19576747,  1.07099097,
          4.64145411, 31.09361762,  9.33476277],
        [ 5.73471425, 40.50536308, 10.37845248,  0.89295713,
          4.13234914, 26.86499712, 11.4911668 ],
        [ 6.35158098, 39.84960039, 10.22816059,  0.99800531,
          4.67111668, 29.63062294,  8.2709131 ],
        [ 6.27286045, 39.58154618,  9.74604092,  1.15514282,
          5.15615792, 30.31956003,  7.76869167]],

       [[ 5.99079933, 37.74368695, 10.18217051,  1.12183636,
          4.68706453, 28.50718905, 11.76725328],
        [ 5.86328694, 37.57099653,  9.27467561,  0.98272983,
          4.01556588, 29.35155748, 12.94118773],
        [ 5.62636423, 39.15469958, 10.07058303,  1.02517682,
          4.83226814, 28.39276992, 10.89813827],
        [ 6.08659505, 40.07623438,  9.74145156,  1.00487041,
...
          4.51185529, 28.61068003, 11.91060951],
        [ 6.08350373, 37.13080261,  8.35016547,  0.94187509,
          4.13360502, 26.69690601, 16.66314207],
        [ 5.79758217, 36.66130662, 10.98527112,  1.07904242,
          4.41286259, 28.64999042, 12.41394465],
        [ 6.1091823 , 41.32863119, 10.0492049 ,  1.1052933 ,
          4.63643229, 31.0966306 ,  5.67462541]],

       [[ 5.52704483, 31.17841807,  8.75169498,  0.94069687,
          4.22722104, 27.45729093, 21.91763328],
        [ 6.25054933, 37.41494275,  9.89899695,  1.0552567 ,
          4.91254778, 32.38668956,  8.08101693],
        [ 5.26046986, 38.01840728,  9.70438991,  0.97272244,
          4.32744373, 25.69212703, 16.02443975],
        [ 6.27394068, 42.0143969 , 10.14654472,  0.89797499,
          3.93263582, 28.64022893,  8.09427795],
        [ 5.92312978, 37.58587223,  9.5092106 ,  1.00438747,
          4.34010567, 28.27211596, 13.36517829],
        [ 5.80135037, 44.05021666, 10.18642737,  0.9861211 ,
          4.09943649, 27.47582317,  7.40062484]]])
Coordinates:
    district        int64 20
  * chain           (chain) int64 0 1 2 3 4 5 6 7
  * draw            (draw) int64 0 1 2 3 4 5
    election_type   <U10 'municipale'
  * party_complete  (party_complete) object 'farleft' 'left' ... 'other'

Now we have the proportions for each party, in each district! But we'll also want to map those results on the geography of Paris. To do that, we'll compute the most likely winner in each district, its probability of winning, and the proportion of the votes it's expected to get. This is done by the code below. I want to thank Grégoire David for [his open-source project, france-geojson](https://github.com/gregoiredavid/france-geojson), where I found the geographic shapes of Paris.

In [179]:
winners = pd.DataFrame(share_est.sel(district=4).idxmax(dim="party_complete").to_series().value_counts(normalize=True).multiply(100).round().astype(int)).reset_index().loc[[0]]
winners.columns = ["winner", "prob"]
winners["district"] = 4
winner = winners.loc[0, "winner"]
winners

,winner,prob,district
0,center,100,4


In [180]:
samples_winner = share_est.sel(district=4).sel(party_complete=winner)

In [181]:
winners["mean"] = samples_winner.mean().data
winners["low"] = az.hdi(samples_winner).sel(hdi="lower")["x"].data
winners["high"] = az.hdi(samples_winner).sel(hdi="higher")["x"].data
winners

,winner,prob,district,mean,low,high
0,center,100,4,80.758174,78.58584,83.029083


In [22]:
win_summary = []
for district in share_est.district.values:
    winners = pd.DataFrame(
        share_est.sel(district=district).idxmax(dim="party_complete").to_series().value_counts(normalize=True).multiply(100).round().astype(int)
    ).reset_index().loc[[0]]
    winners.columns = ["winner", "prob"]
    winners["district"] = district
    
    winner = winners.loc[0, "winner"]
    samples_winner = share_est.sel(district=district).sel(party_complete=winner)
    
    winners["mean"] = samples_winner.mean().data
    winners["low"] = az.hdi(samples_winner).sel(hdi="lower")["x"].data
    winners["high"] = az.hdi(samples_winner).sel(hdi="higher")["x"].data
    
    win_summary.append(winners)
win_summary = pd.concat(win_summary, ignore_index=True)
win_summary

,winner,prob,district,mean,low,high
0,left,100,4,46.449301,42.976895,49.958752
1,left,100,5,48.257724,44.922359,51.499444
2,right,72,6,33.834203,26.268262,41.486825
3,farright,52,7,28.973420,25.903385,32.016798
4,farright,98,8,31.613352,27.444590,35.485433
5,left,100,9,50.433485,47.287162,53.373445
6,left,100,10,58.735258,53.021391,64.537269
7,left,100,11,49.621188,46.465353,52.595341
8,left,99,12,34.862847,31.154535,38.599713
9,left,100,13,47.966889,43.955225,51.819682


In [9]:
win_summary = {}
for district in range(Ndistricts):
    district_shares = share_est[district]
    district_means = share_mean[district]
    district_hpds = az.hpd(district_shares)

    # compute winner in each simulation:
    order = np.squeeze(np.argsort(-district_shares, axis=1)[:, :1])
    winners = pd.Series(order)

    probs = pd.DataFrame(
        winners.value_counts(normalize=True).multiply(100).round().astype(int)
    )
    probs.columns = ["odds"]
    probs["low"], probs["high"], probs["mean"] = np.nan, np.nan, np.nan

    for p_id in probs.index:
        probs.loc[p_id, "low"], probs.loc[p_id, "high"] = district_hpds[p_id]
        probs.loc[p_id, "mean"] = district_means[p_id]
        # assign means to color bins:
        bins_idx = np.digitize(probs.loc[p_id, "mean"], BINS)
        probs.loc[p_id, "color"] = COLORS[PARTIES[p_id]][bins_idx]
    probs.index = np.array(PARTIES)[probs.index]
    probs.index.name = "winner"
    win_summary[district] = probs.reset_index()

win_summary = pd.concat(win_summary).reset_index(level=1)
# keep only most probable winner:
win_summary = (
    win_summary[win_summary.level_1 == 0].drop("level_1", axis=1).set_index(districts)
)

In [10]:
paris_shape = (
    gpd.read_file("../data/paris_shape.json")
    .sort_values("code")
    .reset_index(drop=True)
    .set_index(districts)
)
paris_shape = pd.concat([paris_shape, win_summary], axis=1).reset_index()
paris_shape["winner"] = paris_shape["winner"].str.title()

try: 
    os.remove("oos_data/paris_shape.geojson")
except FileNotFoundError:
    pass
paris_shape.to_file("oos_data/paris_shape.geojson", driver='GeoJSON')
paris_shape.head()

,index,code,nom,geometry,winner,odds,low,high,mean,color
0,1,75101,Paris 1er Arrondissement,"POLYGON ((2.32576 48.86955, 2.32787 48.86986, ...",Left,93,26.010501,34.002147,29.916126,#c994c7
1,2,75102,Paris 2e Arrondissement,"POLYGON ((2.35084 48.86334, 2.32787 48.86986, ...",Left,86,26.299702,34.907635,30.598070,#c994c7
2,3,75103,Paris 3e Arrondissement,"POLYGON ((2.35009 48.86195, 2.35084 48.86334, ...",Left,100,32.922831,42.475738,37.693623,#df65b0
3,4,75104,Paris 4e Arrondissement,"POLYGON ((2.34456 48.85399, 2.35009 48.86195, ...",Left,98,30.081752,38.873754,34.412202,#c994c7
4,5,75105,Paris 5e Arrondissement,"POLYGON ((2.34456 48.85399, 2.36432 48.84617, ...",Left,95,30.592591,39.535722,34.843792,#c994c7


We've now stored all the data we need -- it's time to plot them and see what the model is telling us about the coming elections... See you in the notebook `munic_model_analysis.ipynb` or, even better, [here](https://mybinder.org/v2/gh/AlexAndorra/pollsposition_models/master?urlpath=%2Fvoila%2Frender%2Fdistrict-level%2Fmunic_model_analysis.ipynb)! Indeed, this notebook is designed to be seen online, not really to be read on GitHub, where the map's JavaScript is not displayed.

In [90]:
%load_ext watermark
%watermark -a AlexAndorra -n -u -v -iv

pymc3     3.9.3
arviz     0.9.0
numpy     1.19.1
xarray    0.16.0
pandas    1.0.5
geopandas 0.8.1
AlexAndorra 
last updated: Tue Sep 29 2020 

CPython 3.8.5
IPython 7.17.0
